In [ ]:
# some observations
# imputation by interpolate performs better than other methods like median amd mode
#XGB performs better than RF
# scaling the variables does not improve the performance
# Important features are derived for each output after applying the XGB model
# Grid search for best parameters take multiple days??
#-----------------------
#get the common important features derived for both outputs from XGB model and use them to 
#create a Recurrent Neural Network, LSTM

### Import all the Libraries 

In [2]:
#import all the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.multioutput import MultiOutputRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
import math
import time

### PEM dataset for the prediction of ['NOX_HMI', 'CO_HMI']

In [3]:
# read the respective files from these locations
data_file_training='/data-restricted/PEM/L0162_20200909T0001_20201201T1230.csv'

df = pd.read_csv(data_file_training)
print(df.shape)
print(list(df.columns))

(66331, 46)
['Timestamp', 'BRNDMD', 'WFPILM', 'F_FARFLI', 'F_FARFLO', 'F_FARFL', 'F_W2', 'F_WA36', 'F_WB3Q', 'F_WF36DMD', 'P2SEL', 'PS3SEL', 'T2SEL', 'T3SEL', 'TFLCYCS', 'TFLPDFS', 'TFLIDFS', 'TFLODFS', 'WFINRM', 'WFOTRM', 'WFQPERRCOR', 'A90RH', 'F_WAFL', 'NGGSEL', 'PX36SEL', 'PX36AVAL', 'PX36BVAL', 'NPTSEL', 'NPTREF', 'DWB36PCT', 'REGULATOR', 'T48SEL', 'CDPSEL', 'T8SEL', 'P48SEL', 'LHVSEL', 'SGSEL', 'NOX_HMI', 'CO_HMI', 'O2_HMI', 'VLVFBKG_06', 'EMISSIONS_SS', 'SSFILTER', 'F_WB3', 'NOX15', 'CO15']


In [4]:
#output_columns=['NOX_HMI', 'CO_HMI'] #compare your predictions with 'NOX15', 'CO15'
df['Timestamp']=pd.to_datetime(df['Timestamp'])
df=df.set_index('Timestamp')

### Missing values imputation 

In [5]:
#imputation for the missing values
cols_with_NaNs=df.columns[df.isnull().any()].tolist()
df_NaNs=df[cols_with_NaNs]
percent_missing=df_NaNs.isnull().sum() * 100/len(df_NaNs)
#since 75% of this variable are NaNs
df.drop(('EMISSIONS_SS'),axis=1,inplace=True)
#imputation by interpolate method
df['PX36AVAL']=df['PX36AVAL'].interpolate(method='linear')
df['PX36BVAL']=df['PX36BVAL'].interpolate(method='linear')
#There are no null values

In [6]:
df.drop(['NOX15','CO15'],axis=1,inplace=True)

### Data split 

In [7]:
# will split the data to 70:30 for training and testing
#0.70*66327
train_df,test_df=df[:46428],df[46428:]
print(train_df.shape,test_df.shape)
OUTPUT_COLUMNS=['NOX_HMI','CO_HMI']
train_X=train_df.drop(OUTPUT_COLUMNS,axis=1)
train_y=train_df[OUTPUT_COLUMNS]
test_X=test_df.drop(OUTPUT_COLUMNS,axis=1)
test_y=test_df[OUTPUT_COLUMNS]
print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)

(46428, 42) (19903, 42)
(46428, 40) (46428, 2) (19903, 40) (19903, 2)


### MultiOutputRegressor model as the wrapper for XGBRegressor 

In [8]:
other_params={'learning_rate':0.1,'n_estimators':300,'max_depth':5,'min_child_weight':1,'subsample':0.8,'colsample_bytree':0.8}
multioutputregressor=MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror',**other_params)).fit(train_X,train_y)
predictions=multioutputregressor.predict(test_X)

### Evaluation of the test set

In [9]:
# Evaluation forthe test set_both CO and NOX
print('Root_mean_squared_error_test set_all',math.sqrt(mean_squared_error(test_y,predictions)))
print('R2 Score_test set_all',r2_score(test_y,predictions))

Root_mean_squared_error_test set_all 4.427835662198557
R2 Score_test set_all 0.9330317996353548


In [10]:
# Individual evaluations
print('Root_mean_squared_error_test set_NOX',math.sqrt(mean_squared_error(test_y['NOX_HMI'],predictions[:,0])))
print('R2 Score_test set_NOX',r2_score(test_y['NOX_HMI'],predictions[:,0]))

print('Root_mean_squared_error_test set_CO_HMI',math.sqrt(mean_squared_error(test_y['CO_HMI'],predictions[:,1])))
print('R2 Score_test set_CO_HMI',r2_score(test_y['CO_HMI'],predictions[:,1]))

Root_mean_squared_error_test set_NOX 3.0489351088192436
R2 Score_test set_NOX 0.9826639673759708
Root_mean_squared_error_test set_CO_HMI 5.469501988763151
R2 Score_test set_CO_HMI 0.8833996318947388


### Feature Importance

In [11]:
print(len(multioutputregressor.estimators_))

NOX_HMI=multioutputregressor.estimators_[0].feature_importances_
CO_HMI=multioutputregressor.estimators_[1].feature_importances_
feature_importances_NOX_HMI=pd.DataFrame(NOX_HMI,index=train_X.columns,columns=['importance']).sort_values('importance',ascending=False)
feature_importances_CO_HMI=pd.DataFrame(CO_HMI,index=train_X.columns,columns=['importance']).sort_values('importance',ascending=False)

feature_importances_CO_HMI.index[0:30]
feature_importances_NOX_HMI.index[0:30]

2


Index(['O2_HMI', 'NPTSEL', 'PX36AVAL', 'TFLIDFS', 'F_FARFL', 'PX36BVAL',
       'TFLCYCS', 'T48SEL', 'PX36SEL', 'T8SEL', 'SGSEL', 'F_WB3', 'LHVSEL',
       'TFLPDFS', 'P2SEL', 'F_WA36', 'WFPILM', 'TFLODFS', 'NGGSEL',
       'F_WF36DMD', 'NPTREF', 'SSFILTER', 'T2SEL', 'P48SEL', 'T3SEL', 'F_WB3Q',
       'PS3SEL', 'WFQPERRCOR', 'BRNDMD', 'F_FARFLO'],
      dtype='object')

In [12]:
# common_important_features from top 30 features of both output columns
common_important_features=list(set(feature_importances_CO_HMI.index[0:30]) & set(feature_importances_NOX_HMI.index[0:30]))
print(len(common_important_features),common_important_features)


25 ['TFLIDFS', 'T8SEL', 'F_WB3Q', 'F_FARFL', 'O2_HMI', 'TFLPDFS', 'TFLCYCS', 'NPTREF', 'PX36AVAL', 'P48SEL', 'NGGSEL', 'BRNDMD', 'F_WB3', 'WFPILM', 'PS3SEL', 'SGSEL', 'F_WA36', 'F_WF36DMD', 'WFQPERRCOR', 'LHVSEL', 'TFLODFS', 'P2SEL', 'NPTSEL', 'T3SEL', 'T48SEL']


In [ ]:

### Grid search to get the best parameters 

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
gsc = GridSearchCV(
            estimator=xgb.XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "subsample": [0.8,0.9, 1.0],
                        "colsample_bytree":[ 0.8, 0.9],},
            cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
start1=time.time()
grid_result = MultiOutputRegressor(gsc).fit(train_X, train_y)
end1=time.time()
print('time elapsed: ' + str(end1-start1))

y_predict_gsc = grid_result.predict(test_X)
print('Root_mean_squared_error_test set',math.sqrt(mean_squared_error(test_y,y_predict_gsc)))
print('R2 Score_test set',r2_score(test_y,y_predict_gsc))
